In [1]:
# Dataset - https://www.kaggle.com/datasets/salader/dogs-vs-cats

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#Dataset size is too high. Instead of exporting, adding the Kaggle API to directly import the data

In [3]:
!kaggle datasets download -d salader/dogs-vs-cats
#Downloading dataset from Kaggle

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


The Dataset is downloaded but in zip fotmat.
#Extracting the data by inporting zipfile directory

In [4]:
import zipfile as zp
zip_ref = zp.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

#Importing necessary Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [ ]:
# Creating training and validation datasets using image_dataset_from_directory
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

As there is large size data to train, we have distributed them by Keras Generator named 'image_dataset_from_directory'

Normalizing the image pixel data, to get the data size from 0 to 1...by dividing it with 255

In [ ]:
# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

#Training the CNN Model for Classification

In [ ]:
# create CNN model

model = Sequential()

#First Layer with 32 Filters
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

#Second Layer with 64 Filters
model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

#Third Layer with 128 Filters
model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

Displaying the model summary

In [ ]:
model.summary()

Compiling the model

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# Training the model

In [ ]:
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

# Plotting training and validation accuracy

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

# Plotting training and validation loss

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
#Importing OpenCV for image processing
import cv2

In [ ]:
# Defining the model_predict function to classify the image
# Classifying the new test image
def model_predict(img):
    img = cv2.resize(img, (256, 256))  # Resize the image to 256x256 pixels
    img = img.reshape((1, 256, 256, 3))  # Reshape the image to match model input
    prediction = model.predict(img)  # Make prediction using the model
    if prediction > 0.5:  # Check if the prediction is more than 0.5 (dog)
        print('This is the image of a Dog')
    else:  # If the prediction is 0.5 or less (cat)
        print('This is the image of a Cat')

In [ ]:
# Reading another test image
sampleimg1 = cv2.imread('ABCD_cat.jpg')
model_predict(sampleimg1)

In [ ]:
sampleimg2 = cv2.imread('cat.jpg')
model_predict(sampleimg2)

In [ ]:
sampleimg3 = cv2.imread('Dog.jpeg')
model_predict(sampleimg3)